In [ ]:
# ============================================
# Section 2.1 – DATASET DESCRIPTION AND ANALYSIS
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 100)
plt.style.use('seaborn-v0_8-whitegrid')

# Load the Ames Housing dataset
df = pd.read_csv("AmesHousing.csv")

print("Dataset shape:", df.shape)
df.head()

In [ ]:
# Overview of data types and missing values
print("\n--- Data Info ---")
df.info()

print("\n--- Missing Values ---")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing.head(10)

In [ ]:
# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Numeric features: {len(numeric_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

In [ ]:
target = "SalePrice"
print("\nTarget Summary:")
print(df[target].describe())

plt.figure(figsize=(7,4))
sns.histplot(df[target], kde=True, color='skyblue')
plt.title("Distribution of Sale Price")
plt.show()

print("Skewness:", df[target].skew())

In [ ]:
corr = df.corr(numeric_only=True)[target].sort_values(ascending=False)
print("\nTop 10 Correlated Features with SalePrice:")
print(corr.head(10))

plt.figure(figsize=(6,4))
corr.head(10).drop(target).plot(kind='barh', color='lightgreen')
plt.title("Top Correlated Features with SalePrice")
plt.xlabel("Correlation")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(x='Overall Qual', y='SalePrice', data=df, hue='Overall Qual', palette='Blues', legend=False)
plt.title("Sale Price vs Overall Quality")
plt.show()

plt.figure(figsize=(10,4))
sns.boxplot(x='Neighborhood', y='SalePrice', data=df)
plt.xticks(rotation=45, ha='right')
plt.title("Sale Price by Neighborhood")
plt.show()

In [ ]:
print("\nEDA Summary:")
print(f"- Total records: {df.shape[0]} | Total features: {df.shape[1]}")
print(f"- {len(categorical_cols)} categorical, {len(numeric_cols)} numeric features.")
print(f"- {missing.count()} features contain missing values.")
print(f"- SalePrice is right-skewed ({round(df[target].skew(),2)}).")
print("- Strongly correlated features include OverallQual, GrLivArea, and GarageCars.")

In [ ]:
# ============================================
# SECTION 2.2 – DATA PRE-PROCESSING AND CLEANING
# ============================================

# Check total missing values again
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing.head(10)

# Fill numeric columns with median, categorical with mode
for col in df.columns:
    if df[col].dtype == "object":
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value)
    else:
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)

# Confirm no missing values remain
print("Remaining missing values:", df.isnull().sum().sum())

In [ ]:
# Detect outliers using IQR for key numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in ["Lot Area", "Gr Liv Area", "SalePrice"]:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] > upper, upper,
                np.where(df[col] < lower, lower, df[col]))

# Plot
plt.figure(figsize=(6,4))
sns.boxplot(x=df["SalePrice"], color='lightblue')
plt.title("Boxplot after Outlier Capping (SalePrice)")
plt.show()

In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Apply one-hot encoding for categorical features
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print("New dataset shape after encoding:", df_encoded.shape)
df_encoded.head(3)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = df_encoded.drop("SalePrice", axis=1)
y = df_encoded["SalePrice"]

# Apply standardization
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Check first few scaled values
X_scaled.head(3)

In [ ]:
# Combine scaled features and target for later use
cleaned_df = pd.concat([X_scaled, y], axis=1)

# Save to new CSV
cleaned_df.to_csv("AmesHousing_Cleaned.csv", index=False)
print("Cleaned dataset saved successfully!")

In [ ]:
# ============================================
# SECTION 2.3 – FEATURE ENGINEERING
# ============================================

from lightgbm import LGBMRegressor

# Load the cleaned dataset produced in Section 2.2
df_encoded = pd.read_csv("AmesHousing_Cleaned.csv")

print("Loaded cleaned dataset:", df_encoded.shape)
df_encoded.head(3)

In [ ]:
# Create new numerical features from existing columns
df_encoded["TotalSF"] = df_encoded["Total Bsmt SF"] + df_encoded["Gr Liv Area"]
df_encoded["TotalBath"] = (
    df_encoded.get("Full Bath", 0)
    + 0.5 * df_encoded.get("Half Bath", 0)
    + df_encoded.get("Bsmt Full Bath", 0)
    + 0.5 * df_encoded.get("Bsmt Half Bath", 0)
)
df_encoded["AgeOfHouse"] = 2025 - df_encoded["Year Built"]

# Verify creation
df_encoded[["TotalSF", "TotalBath", "AgeOfHouse"]].head()

In [ ]:
# Split features and target
X = df_encoded.drop("SalePrice", axis=1)
y = df_encoded["SalePrice"]

# Train LightGBM
lgb = LGBMRegressor(random_state=42)
lgb.fit(X, y)

# Feature importance plot
importance = pd.Series(lgb.feature_importances_, index=X.columns).sort_values(ascending=False)
importance.head(15).plot(kind='barh', figsize=(7,5), color='lightgreen')
plt.title("Top 15 Important Features (LightGBM)")
plt.xlabel("Importance")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Select top 20 most important predictors
selected_features = importance.head(20).index.tolist()
X_selected = df_encoded[selected_features]
y = df_encoded["SalePrice"]

# Print selected features 5 per row
print("Selected features for modelling:")
for i in range(0, len(selected_features), 5):
    row = selected_features[i:i+5]
    print(", ".join(row))

print("\nX_selected shape:", X_selected.shape)

In [ ]:
# ============================================
# SECTION 2.4 – MODEL IMPLEMENTATION
# ============================================

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define target and features
target = "SalePrice"
X = df_encoded.drop(target, axis=1)
y = df_encoded[target]

# Split data (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape, "| Test size:", X_test.shape)

In [ ]:
# Initialize Elastic Net model
en_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
en_model.fit(X_train, y_train)

# Predictions
y_pred_en = en_model.predict(X_test)

# Evaluate performance
mae_en = mean_absolute_error(y_test, y_pred_en)
rmse_en = np.sqrt(mean_squared_error(y_test, y_pred_en))
r2_en = r2_score(y_test, y_pred_en)

print(f"Elastic Net → MAE: {mae_en:.2f}, RMSE: {rmse_en:.2f}, R²: {r2_en:.3f}")

In [ ]:
# Initialize LightGBM model
lgb_model = LGBMRegressor(random_state=42, n_estimators=300, learning_rate=0.05)
lgb_model.fit(X_train, y_train)

# Predictions
y_pred_lgb = lgb_model.predict(X_test)

# Evaluate performance
mae_lgb = mean_absolute_error(y_test, y_pred_lgb)
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
r2_lgb = r2_score(y_test, y_pred_lgb)

print(f"LightGBM → MAE: {mae_lgb:.2f}, RMSE: {rmse_lgb:.2f}, R²: {r2_lgb:.3f}")

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_en, alpha=0.6, label="Elastic Net")
plt.scatter(y_test, y_pred_lgb, alpha=0.6, label="LightGBM")
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Actual Sale Price")
plt.ylabel("Predicted Sale Price")
plt.title("Actual vs Predicted Sale Price")
plt.legend()
plt.show()

In [ ]:
# ============================================
# SECTION 2.5 – MODEL TUNING AND OPTIMIZATION
# ============================================

from sklearn.model_selection import GridSearchCV

# Define parameter grid for Elastic Net
param_grid_en = {
    'alpha': [0.01, 0.1, 1, 10],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# Initialize model
en = ElasticNet(random_state=42)

# Perform 5-fold cross-validation grid search
grid_en = GridSearchCV(
    estimator=en,
    param_grid=param_grid_en,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid_en.fit(X_train, y_train)

# Display best parameters and score
print("Best ElasticNet parameters:", grid_en.best_params_)
print("Best cross-validated R²:", grid_en.best_score_)

# Evaluate on test set
best_en = grid_en.best_estimator_
y_pred_en_tuned = best_en.predict(X_test)

mae_en_tuned = mean_absolute_error(y_test, y_pred_en_tuned)
rmse_en_tuned = np.sqrt(mean_squared_error(y_test, y_pred_en_tuned))
r2_en_tuned = r2_score(y_test, y_pred_en_tuned)

print(f"Tuned Elastic Net → MAE: {mae_en_tuned:.2f}, RMSE: {rmse_en_tuned:.2f}, R²: {r2_en_tuned:.3f}")


In [ ]:
# Parameter grid for LightGBM
param_grid_lgb = {
    'num_leaves': [20, 31, 40],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [200, 400, 600],
    'max_depth': [6, 8, 10]
}

lgb = LGBMRegressor(random_state=42)

grid_lgb = GridSearchCV(
    estimator=lgb,
    param_grid=param_grid_lgb,
    scoring='r2',
    cv=3,
    n_jobs=-1,
    verbose=1
)
grid_lgb.fit(X_train, y_train)

print("Best LightGBM parameters:", grid_lgb.best_params_)
print("Best cross-validated R²:", grid_lgb.best_score_)

# Evaluate best model
best_lgb = grid_lgb.best_estimator_
y_pred_lgb_tuned = best_lgb.predict(X_test)

mae_lgb_tuned = mean_absolute_error(y_test, y_pred_lgb_tuned)
rmse_lgb_tuned = np.sqrt(mean_squared_error(y_test, y_pred_lgb_tuned))
r2_lgb_tuned = r2_score(y_test, y_pred_lgb_tuned)

print(f"Tuned LightGBM → MAE: {mae_lgb_tuned:.2f}, RMSE: {rmse_lgb_tuned:.2f}, R²: {r2_lgb_tuned:.3f}")

In [ ]:
results = pd.DataFrame({
    'Model': ['ElasticNet (Tuned)', 'LightGBM (Tuned)'],
    'MAE': [mae_en_tuned, mae_lgb_tuned],
    'RMSE': [rmse_en_tuned, rmse_lgb_tuned],
    'R²': [r2_en_tuned, r2_lgb_tuned]
})
print(results)

In [ ]:
# Compute residuals for both models
residuals_en = y_test - y_pred_en_tuned
residuals_lgb = y_test - y_pred_lgb_tuned

# Plot residual distributions
plt.figure(figsize=(8,4))
sns.kdeplot(residuals_en, label='Elastic Net', fill=True)
sns.kdeplot(residuals_lgb, label='LightGBM', fill=True)
plt.title("Residual Error Distribution")
plt.xlabel("Prediction Error (Actual - Predicted)")
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

cv_en = cross_val_score(best_en, X, y, scoring='r2', cv=5)
cv_lgb = cross_val_score(best_lgb, X, y, scoring='r2', cv=5)

print("Elastic Net CV R² scores:", cv_en)
print("LightGBM CV R² scores:", cv_lgb)

print("\nMean CV R²:")
print("Elastic Net:", cv_en.mean())
print("LightGBM:", cv_lgb.mean())

In [ ]:
# ============================================
# SECTION 2.7 – MODEL EXPLAINABILITY
# ============================================

import shap

# Create SHAP explainer for the tuned LightGBM model
explainer = shap.TreeExplainer(best_lgb)
shap_values = explainer.shap_values(X)

# Generate SHAP summary bar plot for top features
shap.summary_plot(shap_values, X, plot_type="bar", max_display=15)


In [ ]:
# Visualize how TotalSF affects the predicted SalePrice
shap.dependence_plot("TotalSF", shap_values, X)